In [1]:
%load_ext autoreload
%autoreload 2

import os
import random
import sys

from pathlib import Path
from hydra import initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
import shutil
import tqdm
import torch
import matplotlib.pyplot as plt

RESULTS_DIR = "singles"
NUM_REPEATS = 3

In [8]:
# New single results
session_ids = list(os.listdir(RESULTS_DIR))

# session_ids = ["MonkeyG_20150918_Session1_M1"]

results = {}
for sid in session_ids:
    # if sid == "MonkeyJ_20160624_Session3_S1":
    #     continue
    rsum = 0
    for nidx in range(NUM_REPEATS):
        sf = os.path.join(RESULTS_DIR, sid, f"results_{nidx}.torch")
        _r = torch.load(sf, map_location=torch.device('cpu'))
        print(_r["test_r2s"])
        _r = max([rr[-1] for rr in _r["test_r2s"]])
        rsum += _r
    results[sid] = rsum / NUM_REPEATS

[(0, -0.00882114376872778), (10000, -0.16370812058448792), (20000, -0.20018553733825684)]
[(0, -0.01036879699677229), (10000, -0.1705847978591919), (20000, -0.19029942154884338)]
[(0, -0.01034440565854311), (10000, -0.1653931438922882), (20000, -0.18839211761951447)]


In [10]:
# Old results
infile = "/home/mmattb/Projects/opto-coproc/multisession_st_dist.torch"
old_results = torch.load(infile, map_location=torch.device('cpu'))

def parse_results(results):
    dist = []
    disttrend = []
    dist_test = []
    disttrend_test = []
    dist_loss = []
    dist_loss_test = []
    per_session_train_r2s = {}
    per_session_test_r2s = {}
    per_session_test_losses = {}
    for k, result in results.items():
        per_session_test_losses[k] = []
        for sr in result:
            dist.append(max(sr[-5]))
            dist_test.append(max([rr[-1] for rr in sr[-2]]))

            disttrend.append(sr[6])
            disttrend_test.append(sr[7])
    
            dist_loss.append(min(sr[2]))
            dist_loss_test.append(min(sr[3]))
    
            per_session_train_r2s[k] = dist[-1]
            per_session_test_r2s[k] = dist_test[-1]
    
            per_session_test_losses[k].append(dist_loss_test[-1])

    return per_session_test_r2s

old_results = parse_results(old_results)

In [9]:
for sid, new_tr2 in results.items():
    old_tr2 = old_results[sid]
    print(sid, old_tr2, new_tr2)

# Bad....

MonkeyG_20150918_Session1_M1 0.7078850865364075 -0.009844782141347727
